In [ ]:
# %%
####################
## ASSIGN REVIEWS ##
####################
# Imports
import numpy as np
import pandas as pd
import duckdb

import sys
sys.path.append("..")
from assign_reviews import create_objective_fun, create_lb_ub, create_constraints, solve_milp, format_and_output_result

# Start script

In [ ]:
mkdir output

In [ ]:
ASSIGN_TUTORIALS_TO_ANYONE = False
TUTORIAL_COEFF = 0.8

DEBUG = True

database_file = "../data/assign_reviews_031024.db"
con = duckdb.connect(database_file)
df_submissions = con.sql("table submissions_to_assign").df()
df_reviewers = con.sql("table reviewers_to_assign").df()

df_submissions = df_submissions.assign(assigned_reviewer_ids=[[]] * len(df_submissions))
df_reviewers = df_reviewers.assign(assigned_submission_ids=[[]] * len(df_reviewers))

len(df_submissions), len(df_reviewers)

In [ ]:
df_submissions[df_submissions.track=="TUT"]

## Assign tutorial reviewers

In [ ]:
MIN_TUTORIALS_PER_PERSON = 0
MAX_TUTORIALS_PER_PERSON = 5
MIN_REVIEWERS_PER_TUTORIAL = 3
MAX_REVIEWERS_PER_TUTORIAL = 4

df_submissions_tutorials = df_submissions[df_submissions.track=="TUT"]

solution = solve_milp(
    df_reviewers,
    df_submissions_tutorials,
    MIN_TUTORIALS_PER_PERSON,
    MAX_TUTORIALS_PER_PERSON,
    MIN_REVIEWERS_PER_TUTORIAL,
    MAX_REVIEWERS_PER_TUTORIAL,
    TUTORIAL_COEFF,
    ASSIGN_TUTORIALS_TO_ANYONE
)
reviewers, submissions = format_and_output_result(df_reviewers, df_submissions_tutorials, solution, post_fix="00")

In [ ]:
df = pd.DataFrame(reviewers)
df

In [ ]:
df_reviewers_with_tut = df_reviewers.assign(assigned_submission_ids=df.assigned_submission_ids)
df_reviewers_with_tut

In [ ]:
con.sql("select * from df_reviewers_with_tut")

In [ ]:
con.sql("create or replace table reviewer_assignments_00 as select * from df_reviewers_with_tut")

In [ ]:
df = pd.DataFrame(submissions)
df

In [ ]:
con.sql("""
create or replace table submission_assignments_00 as
select df_submissions.submission_id, df_submissions.author_ids, df_submissions.track,
list_concat(df_submissions.assigned_reviewer_ids, df.assigned_reviewer_ids) as assigned_reviewer_ids
from df_submissions
left join df on df.submission_id = df_submissions.submission_id
""")
con.sql("table submission_assignments_00")

## Assign talk reviewers

In [ ]:
df_reviewers_with_tut[df_reviewers_with_tut.assigned_submission_ids.apply(len) == 0]

In [ ]:
MIN_REVIEWS_PER_PERSON = 5
MAX_REVIEWS_PER_PERSON = 9
MIN_REVIEWERS_PER_SUBMISSION = 2
MAX_REVIEWERS_PER_SUBMISSION = 4

df_reviewers_no_submissions = df_reviewers_with_tut[df_reviewers_with_tut.assigned_submission_ids.apply(len) == 0]
df_submissions_no_tutorials = df_submissions[df_submissions.track!="TUT"]

solution = solve_milp(
    df_reviewers_no_submissions,
    df_submissions_no_tutorials,
    MIN_REVIEWS_PER_PERSON,
    MAX_REVIEWS_PER_PERSON,
    MIN_REVIEWERS_PER_SUBMISSION,
    MAX_REVIEWERS_PER_SUBMISSION,
    TUTORIAL_COEFF,
    ASSIGN_TUTORIALS_TO_ANYONE
)
if solution is not None:
    reviewers, submissions = format_and_output_result(df_reviewers_no_submissions, df_submissions_no_tutorials, solution, post_fix="01")

In [ ]:
df_reviewers_with_tut

In [ ]:
df = pd.DataFrame(reviewers)[["reviewer_id", "assigned_submission_ids"]]

In [ ]:
df

In [ ]:
con.sql("""
create or replace table reviewer_assignments_01 as
select
    df_reviewers_with_tut.reviewer_id, tracks, conflicts_submission_ids,
    list_concat(df_reviewers_with_tut.assigned_submission_ids, df.assigned_submission_ids) as assigned_submission_ids
from df_reviewers_with_tut
left join df on df.reviewer_id = df_reviewers_with_tut.reviewer_id
""")
con.sql("table reviewer_assignments_01")

In [ ]:
df = pd.DataFrame(submissions)
df

In [ ]:
con.sql("""
create or replace table submission_assignments_01 as
select submission_assignments_00.submission_id, submission_assignments_00.author_ids, submission_assignments_00.track,
list_concat(submission_assignments_00.assigned_reviewer_ids, df.assigned_reviewer_ids) as assigned_reviewer_ids
from submission_assignments_00
left join df on df.submission_id = submission_assignments_00.submission_id
""")
con.sql("table submission_assignments_01")

In [ ]:
df = pd.DataFrame(submissions)
df = df.assign(num_reviewers=df.assigned_reviewer_ids.apply(len))
df[df.num_reviewers>2]
df[df.num_reviewers==2]

## Assign talks to tutorial reviewers

In [ ]:
df = pd.DataFrame(submissions)
df = df.assign(num_reviewers=df.assigned_reviewer_ids.apply(len))
df_submissions_few_reviewers = df[df.num_reviewers==2]

In [ ]:
MIN_REVIEWS_PER_PERSON = 0
MAX_REVIEWS_PER_PERSON = 4
MIN_REVIEWERS_PER_SUBMISSION = 1
MAX_REVIEWERS_PER_SUBMISSION = 2

df_reviewers_only_tut = df_reviewers_with_tut[df_reviewers_with_tut.assigned_submission_ids.apply(len) > 0]

solution = solve_milp(
    df_reviewers_only_tut,
    df_submissions_few_reviewers,
    MIN_REVIEWS_PER_PERSON,
    MAX_REVIEWS_PER_PERSON,
    MIN_REVIEWERS_PER_SUBMISSION,
    MAX_REVIEWERS_PER_SUBMISSION,
    TUTORIAL_COEFF,
    ASSIGN_TUTORIALS_TO_ANYONE
)

if solution is not None:
    reviewers, submissions = format_and_output_result(df_reviewers_only_tut, df_submissions_few_reviewers, solution, post_fix="02")

In [ ]:
df = pd.DataFrame(submissions)
df = df.assign(num_reviewers=df.assigned_reviewer_ids.apply(len))
df

In [ ]:
df = pd.DataFrame(reviewers)
df = df[["reviewer_id", "assigned_submission_ids"]]
df

In [ ]:
con.sql("""
create or replace table reviewer_assignments_02 as
select
    reviewer_assignments_01.reviewer_id, tracks, conflicts_submission_ids,
    list_concat(reviewer_assignments_01.assigned_submission_ids, df.assigned_submission_ids) as assigned_submission_ids
from reviewer_assignments_01
left join df on df.reviewer_id = reviewer_assignments_01.reviewer_id
""")
con.sql("table reviewer_assignments_02")

In [ ]:
con.sql("select count(*), string_agg(reviewer_id), len(assigned_submission_ids) as num_submissions from reviewer_assignments_02 group by num_submissions")

In [ ]:
df = pd.DataFrame(submissions)
df

In [ ]:
con.sql("""
create or replace table submission_assignments_02 as
select submission_assignments_01.submission_id, submission_assignments_01.author_ids, submission_assignments_01.track,
list_concat(submission_assignments_01.assigned_reviewer_ids, df.assigned_reviewer_ids) as assigned_reviewer_ids
from submission_assignments_01
left join df on df.submission_id = submission_assignments_01.submission_id
""")
con.sql("table submission_assignments_02")

## Final counts/checks

All submissions have at least 3 reviewers

In [ ]:
con.sql("""
select string_agg(submission_id), count(track), len(assigned_reviewer_ids) from submission_assignments_02 group by len(assigned_reviewer_ids)
""")

Step 1: Only tutorial assignments

In [ ]:
con.sql("""
select string_agg(reviewer_id), count(reviewer_id), string_agg(tracks), len(assigned_submission_ids) from reviewer_assignments_00 group by len(assigned_submission_ids)
""")

Step 2: Add talks assignments

In [ ]:
con.sql("""
select string_agg(reviewer_id), count(reviewer_id), string_agg(tracks), len(assigned_submission_ids) from reviewer_assignments_01 group by len(assigned_submission_ids)
""")

Step 3: Assign talks to tutorial reviewers

In [ ]:
con.sql("""
select string_agg(reviewer_id), count(reviewer_id), string_agg(tracks), len(assigned_submission_ids) from reviewer_assignments_02 group by len(assigned_submission_ids)
""")

In [ ]:
con.close()

## Final export

In [ ]:
import duckdb
database_file = "../data/assign_reviews_031024.db"
con = duckdb.connect(database_file)

In [ ]:
import json
reviewer_assignments_final = {
    item["reviewer_id"]: item["assigned_submission_ids"]
    for item in
    con.sql("table reviewer_assignments_02").df()[["reviewer_id", "assigned_submission_ids"]].to_dict("records")
}
with open(f"output/reviewer-assignments.json", "w") as fp:
        fp.write(json.dumps(reviewer_assignments_final, indent=4))

In [ ]:
con.close()